In [59]:
import pandas as pd
import requests
import pandas as pd
import json
import re

data = pd.read_csv('clean_data.csv')
train_set = pd.read_csv('train_data_pajak.csv')
train_set.head(10)

,NAMA_BARANG,JML_BARANG,JENIS
0,ANALISA SAMPEL RBD PALM STEARIN,1.0,service
1,Analisa Pengujian INSPEKSI MINYAK KELAPA SAWI...,1.0,service
2,Discarging Survey of CPO 2999998 MT at Kabil B...,1.0,service
3,Discarging Survey of CPO 3149193 MT at Kabil B...,1.0,service
4,KALIBRASI SOUNDING TAPE CPO,1.0,service
5,Loading Survey of CPKO 1999887 MT and CPO 2503...,1.0,service
6,CPOCRUDE PALM OIL,93.0,product
7,VITAMIN A PALMITATE 17 mIUg 0418579294 FO HS 2...,250.0,product
8,Lebih Angkut Palm Kernel,378.0,service
9,CPOCrude Palm Oil Kontrak No082XXXXXX20XX,410.0,product


In [ ]:
data.head()

In [70]:
def run_curl_command(prompt):
    url = "http://108.137.109.168:11434/api/generate"

    data = {
        "model": "Classsifier1",
        "prompt": prompt,
        "stream": False
    }

    # Sending a POST request
    response = requests.post(url, json=data)

    # Checking the response
    if response.status_code == 200:
        # Return the JSON response
        return response.json()
    else:
        print(f"Error! Status code: {response.status_code}, Response: {response.text}")
        # Return None or raise an exception, depending on your preference
        return None

In [71]:
#
# test the result
jsonRespond = run_curl_command("What color is the sky at different times of the day? Respond using JSON")
# jsonRespond_res = json.loads(jsonRespond['response'])
# print(jsonRespond_res['6:00 AM']['color'])
# print(jsonRespond_res['6:00 AM']['description'])

In [52]:
print(jsonRespond)

{'model': 'mistral', 'created_at': '2024-02-22T09:53:39.625627033Z', 'response': ' {\n "6:00 AM": {\n   "color": "#F0F8FF",\n   "description": "Sky is usually light blue or white, depending on weather conditions"\n },\n "9:00 AM": {\n   "color": "#BFEFFF",\n   "description": "Sky gradually turns lighter blue as the sun rises higher"\n },\n "12:00 PM": {\n   "color": "#9FC5DE",\n   "description": "Clear blue sky in the afternoon"\n },\n "3:00 PM": {\n   "color": "#ADD8E6",\n   "description": "Sky may have a hint of pink or orange near the horizon"\n },\n "6:00 PM": {\n   "color": "#C5CAE9",\n   "description": "Sky begins to change color as sun sets, usually appearing pink, orange, purple or red"\n },\n "9:00 PM": {\n   "color": "#47484B",\n   "description": "Night sky is typically dark blue or black with stars visible"\n }\n}', 'done': True, 'context': [733, 16289, 28793, 28705, 1824, 3181, 349, 272, 7212, 438, 1581, 2421, 302, 272, 1370, 28804, 1992, 19571, 1413, 9292, 733, 28748, 1628

In [72]:
prompt = """
Please make classification for below text string based on the following criteria
1. type represents the type of classification ('Service' or 'Product'). If the product contains a verb or various activities or offerings that support the production, distribution, marketing, or utilization of palm oil and its derivatives, classify it as 'service'.If the product is raw material or a product typically refers to the tangible goods derived from palm oil classify it as 'product'.
2. confidenceLevel represents the confidence level of the classification as real number between 0.00 and 1.00
3. explanation provides additional information about the classification.
4. quantity as the quantity stated in the text, please put 0 if you cant find it, and dont put other explanation outside json structure
5. Do not add any text outside the json structure
This is the text to classify:
{y}

Respond using JSON
"""

In [73]:
print(prompt.format(y=data['NAMA_BARANG'].loc[0]))


Please make classification for below text string based on the following criteria
1. type represents the type of classification ('Service' or 'Product'). If the product contains a verb or various activities or offerings that support the production, distribution, marketing, or utilization of palm oil and its derivatives, classify it as 'service'.If the product is raw material or a product typically refers to the tangible goods derived from palm oil classify it as 'product'.
2. confidenceLevel represents the confidence level of the classification as real number between 0.00 and 1.00
3. explanation provides additional information about the classification.
4. quantity as the quantity stated in the text, please put 0 if you cant find it, and dont put other explanation outside json structure
5. Do not add any text outside the json structure
This is the text to classify:
ANALISA SAMPEL RBD PALM STEARIN

Respond using JSON



In [74]:
jsRespond = run_curl_command(prompt.format(y=data['NAMA_BARANG'].loc[0]))

In [75]:
jsClass = json.loads(jsRespond['response'])

In [76]:
# jsClass['type']
jsClass

{'type': 'Product',
 'confidenceLevel': 1.0,
 'explanation': 'RBD palm stearin is a product derived from palm oil.',
 'quantity': 1.0}

In [77]:
lsResult = []
lstError = []

from tqdm import tqdm

for index in tqdm(range(0, 7)):
    try:
        jsRespond = run_curl_command(prompt.format(y=data['NAMA_BARANG'].loc[index]))
        jsClass = json.loads(jsRespond['response'])
        lsResult.append(jsClass)
        
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON for index {index}: {e}")
        # Append the error to lstError
        lstError.append(f"Error decoding JSON for index {index}: {e}")
        # Continue with the next iteration of the loop
        continue
        
    except Exception as ex:
        print(f"An unexpected error occurred for index {index}: {ex}")
        # Append the error to lstError
        lstError.append(f"Unexpected error for index {index}: {ex}")
        # Continue with the next iteration of the loop
        continue


100%|██████████| 7/7 [00:09<00:00,  1.34s/it]


In [ ]:
print(lstError)

In [79]:
#
#
#
# Full scale operation
#
#

from tqdm import tqdm
import json
from datetime import datetime

lsResult = []
lstError = []

for index in tqdm(range(len(data))):
    try:
        jsRespond = run_curl_command(prompt.format(y=data['NAMA_BARANG'].loc[index]))
        jsClass = json.loads(jsRespond['response'])
        lsResult.append(jsClass)
    except json.JSONDecodeError as e:
        error_message = f"Error decoding JSON for index {index} with data '{data['NAMA_BARANG'].loc[index]}': {e}\nOriginal text: {jsRespond['response']}"
        print(error_message)
        # Append the error message to lstError
        lstError.append(error_message)
        # Continue with the next iteration of the loop
        continue
    except Exception as ex:
        error_message = f"Unexpected error for index {index} with data '{data['NAMA_BARANG'].loc[index]}': {ex}"
        print(error_message)
        # Append the error message to lstError
        lstError.append(error_message)
        # Continue with the next iteration of the loop
        continue

# Write lsResult to result.txt with a timestamp prefix
result_filename = f"result_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
with open(result_filename, 'w') as result_file:
    for result in lsResult:
        result_file.write(json.dumps(result) + '\n')

# Write lstError to error.txt with a timestamp prefix
error_filename = f"error_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
with open(error_filename, 'w') as error_file:
    for error_message in lstError:
        error_file.write(error_message + '\n')

# result.txt and error.txt with timestamped filenames containing the contents of lsResult and lstError.


100%|██████████| 1532/1532 [23:30<00:00,  1.09it/s]


In [23]:
jsRespond = run_curl_command(prompt.format(y="10 Bl Blanko Analisa Minyak Sawit Oil Loss SCPR  0010810374"))

In [24]:
jsClass = json.loads(jsRespond['response'])

In [11]:
testInput = """
{
"type": "product",
"confidenceLevel": 0.95,
"explanation": "The text 'Master Key Gembok Truk CPO' does not contain any verbs or activities related to the production, distribution, marketing, or utilization of palm oil and its derivatives. Instead, it appears to be referring to a product or key made from CPO (Crude Palm Oil), which is a raw material derived from palm oil. Therefore, it is classified as a 'product'."
}
"""

In [12]:
json.loads(testInput)

{'type': 'product',
 'confidenceLevel': 0.95,
 'explanation': "The text 'Master Key Gembok Truk CPO' does not contain any verbs or activities related to the production, distribution, marketing, or utilization of palm oil and its derivatives. Instead, it appears to be referring to a product or key made from CPO (Crude Palm Oil), which is a raw material derived from palm oil. Therefore, it is classified as a 'product'."}